In [5]:
 # libs
import pandas as pd
import numpy as np

# data
PII_hearttable = pd.read_csv('../data/dev_sample_HeartTable_10_17_2022 - Copy.csv')
agr_demo = pd.read_csv('../data/cf_agrAppend_va2_10_14_2022 - Copy.csv')
acxiom_demo = pd.read_csv('../data/cf_acxiomAppend_va2_10_14_2022 - Copy.csv')

# ignore for now until ik how i want to add it to the equation
appriss_crime = pd.read_csv('../data/appriss_crime_cleaned_CFconfidenceapplied_v2 - Copy.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_4972\715167181.py:7: DtypeWarning: Columns (12,66) have mixed types. Specify dtype option on import or set low_memory=False.
  agr_demo = pd.read_csv('../data/cf_agrAppend_va2_10_14_2022 - Copy.csv')


In [ ]:
# need to break data in 5 cdc categories


In [ ]:
import json

def calculate_odds_ratios(df, class_column, columns):
    # Initialize an empty dictionary to hold the results
    results = {}

    # Loop through the specified columns
    for column in columns:
        # Initialize a dictionary to hold the results for this column
        results[column] = {}

        # Get the unique values in the column
        unique_values = df[column].dropna().unique()

        # Calculate the percentage of null value for each column
        null_percentages = df[column].isnull().mean()

        # Calculate the threshold for inclusion in the risk score
        threshold = (1.0 / (len(unique_values))) * (1 - null_percentages)

        # Loop through the unique values
        for value in unique_values:
            # Initialize a dictionary to hold the results for this value
            results[column][value] = {}

            # Calculate the proportions and odds for each class, scaling by the null percentages
            class_true = df[df[class_column] == True]
            class_false = df[df[class_column] == False]

            prop_true = (class_true[column] == value).sum() / len(class_true)
            prop_false = (class_false[column] == value).sum() / len(class_false)

            odds_true = prop_true / (1 - prop_true)
            odds_false = prop_false / (1 - prop_false)

            # Calculate the odds ratio
            odds_ratio = odds_true / odds_false

            # Calculate the percentage of data that comprises a given value
            percentage = (df[column] == value).sum() / len(df)

            # Add the results to the dictionary
            results[column][value]['odds_ratio'] = odds_ratio
            results[column][value]['percentage'] = "{:.2%}".format(percentage)
            # add the threshold to the dictionary
            results[column][value]['threshold'] = "{:.2%}".format(threshold)

    # Convert the results to a JSON-like string
    results_json = json.dumps(results, indent=4)

    # Return the JSON-like string
    return results_json

results_json = calculate_odds_ratios(agr_demo, 'important_class', ['DEMO_OWN OR RENT', "DEMO_TYPE OF DWELLING", "RESIDENTIAL", "PROPERTY TYPE DESC", "DEMO_MARITAL STATUS"])
print(results_json)